In [1]:
from sqlalchemy import create_engine, text, bindparam, Integer
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [15]:
class ArticuloEspesor:
    def __init__(self):
        self.server_solmicro = r'srvsql'
        self.server_industry = r'SERVIDOR'
        self.database_solmicro = 'SolmicroERP6_PruebasSub'
        self.database_industry = 'IPFavram'
        self.username_solmicro = 'sa'
        self.password_solmicro = 'Altai2021'
        self.password_industry = '71zl6p9h'
        self.connection_string_solmicro = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_solmicro}@{self.server_solmicro}/{self.database_solmicro}?driver=SQL+Server')
        self.connection_string_industry = create_engine(
            f'mssql+pyodbc://{self.username_solmicro}:{self.password_industry}@{self.server_industry}/{self.database_industry}?driver=SQL+Server')
        self.connection = None
        self.connection_industry = None
        self.connection_solmicro = f'DRIVER={{SQL Server}};SERVER={self.server_solmicro};DATABASE={self.database_solmicro};UID={self.username_solmicro};PWD={self.password_solmicro}'
        self.ruta_archivo = r'ArtiConProveedorIndustry.xlsx' 
        self.hoja_excel = 'Sheet1'  

    def Open_Conn_Solmicro(self):
        try:
            self.connection = self.connection_string_solmicro.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def Open_Conn_Industry(self):
        try:
            self.connection = self.connection_string_industry.connect()
            return self.connection
        except Exception as e:
            print("Error opening connection: ", e)

    def getArtFromIndustry(self):
        resultados = []
        try:
            conn = self.Open_Conn_Industry()
            if conn:
                query = text(f"select CodigoArticulo, Espesor from MArticuloChapas")
                result = conn.execute(query).fetchall()
                if result:
                    resultados.extend(result)
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            if conn:
                conn.close()

    def checkArtInSeolmicro(self,listArticle):
        resultados = []
        try:
            conn = self.Open_Conn_Solmicro()
            if conn:
                for id,espesor in listArticle:
                    query = text(f"SELECT IDArticulo FROM tbMaestroArticulo WHERE (IDArticulo = N'{id}')")
                    result = conn.execute(query).fetchone()
                    if result:
                        updateQuery = text(f"UPDATE tbMaestroArticulo SET Espesor =N'{espesor}' WHERE (IDArticulo = N'{id}')")
                        conn.execute(updateQuery)
                        resultados.append(id)
                conn.commit()
                conn.close()
            return resultados
        except Exception as e:
            print("Error en la consulta:", e)
            return resultados


In [16]:
iEspesor = ArticuloEspesor()

In [17]:
listArtIndustry = iEspesor.getArtFromIndustry()

In [18]:
len(listArtIndustry)

844

In [19]:
checkArti= iEspesor.checkArtInSeolmicro(listArticle=listArtIndustry)

In [9]:
len(checkArti)

844